# Multi-Microgrid Controllers

To actively manipulate multiple microgrids during experiments, we can implement a custom `Controller` that manages several microgrids simultaneously. 

In this example, our controller will coordinate two datacenters by adjusting their power consumption based on their current power deltas and battery states.

In [1]:
import vessim as vs

# Required for running Mosaic in Jupyter notebooks (fixes asyncio event loop conflicts)
import nest_asyncio
nest_asyncio.apply()

The key is implementing the `step()` method, which receives microgrid states as a dictionary. 
This allows the controller to coordinate across multiple microgrids.

In [2]:
from datetime import datetime

class CustomController(vs.Controller):
    def step(self, time: datetime, microgrid_states: dict[str, dict]) -> None:
        berlin = microgrid_states["Berlin"]
        mumbai = microgrid_states["Mumbai"]
        
        print(f"{time.strftime('%H:%M')}: Berlin: {berlin['p_delta']:.0f}W, Mumbai: {mumbai['p_delta']:.0f}W")

Now we'll create two datacenters in different cities and add our controller to monitor them.

In [3]:
environment = vs.Environment(sim_start="2022-06-15", step_size=300)

# Berlin datacenter
berlin = environment.add_microgrid(
    name="Berlin",
    actors=[
        vs.Actor(name="server", signal=vs.ConstantSignal(value=-400)),
        vs.Actor(name="solar", signal=vs.Trace.load("solcast2022_global", column="Berlin", params={"scale": 2000})),
    ],
    storage=vs.SimpleBattery(capacity=300),
)

# Mumbai datacenter  
mumbai = environment.add_microgrid(
    name="Mumbai",
    actors=[
        vs.Actor(name="server", signal=vs.ConstantSignal(value=-350)),
        vs.Actor(name="solar", signal=vs.Trace.load("solcast2022_global", column="Mumbai", params={"scale": 1800})),
    ],
    storage=vs.SimpleBattery(capacity=250),
)

# Add controllers
monitor = vs.Monitor([berlin, mumbai], outdir="./results")
load_balancer = CustomController([berlin, mumbai])

environment.add_controller(monitor)
environment.add_controller(load_balancer)

environment.run(until=12 * 3600)  # 12 hours

2025-07-06 15:24:39.670 | INFO     | mosaik.async_scenario:start:361 - Starting "Actor" as "Berlin.actor.server" ...
2025-07-06 15:24:39.672 | INFO     | mosaik.async_scenario:start:361 - Starting "Actor" as "Berlin.actor.solar" ...
2025-07-06 15:24:39.673 | INFO     | mosaik.async_scenario:start:361 - Starting "Grid" as "Berlin.grid" ...
2025-07-06 15:24:39.674 | INFO     | mosaik.async_scenario:start:361 - Starting "Storage" as "Berlin.storage" ...
2025-07-06 15:24:39.784 | INFO     | mosaik.async_scenario:start:361 - Starting "Actor" as "Mumbai.actor.server" ...
2025-07-06 15:24:39.785 | INFO     | mosaik.async_scenario:start:361 - Starting "Actor" as "Mumbai.actor.solar" ...
2025-07-06 15:24:39.786 | INFO     | mosaik.async_scenario:start:361 - Starting "Grid" as "Mumbai.grid" ...
2025-07-06 15:24:39.787 | INFO     | mosaik.async_scenario:start:361 - Starting "Storage" as "Mumbai.storage" ...
2025-07-06 15:24:39.788 | INFO     | mosaik.async_scenario:start:361 - Starting "Controlle

00:00: Berlin: -400W, Mumbai: -350W
00:05: Berlin: -400W, Mumbai: -350W
00:10: Berlin: -400W, Mumbai: -350W
00:15: Berlin: -400W, Mumbai: -350W
00:20: Berlin: -400W, Mumbai: -350W
00:25: Berlin: -400W, Mumbai: -350W
00:30: Berlin: -400W, Mumbai: -350W
00:35: Berlin: -400W, Mumbai: -350W
00:40: Berlin: -400W, Mumbai: -350W
00:45: Berlin: -400W, Mumbai: -346W
00:50: Berlin: -400W, Mumbai: -341W
00:55: Berlin: -400W, Mumbai: -335W
01:00: Berlin: -400W, Mumbai: -326W
01:05: Berlin: -400W, Mumbai: -316W
01:10: Berlin: -400W, Mumbai: -304W
01:15: Berlin: -400W, Mumbai: -290W
01:20: Berlin: -400W, Mumbai: -277W
01:25: Berlin: -400W, Mumbai: -267W
01:30: Berlin: -400W, Mumbai: -260W
01:35: Berlin: -400W, Mumbai: -253W
01:40: Berlin: -400W, Mumbai: -242W
01:45: Berlin: -400W, Mumbai: -227W
01:50: Berlin: -400W, Mumbai: -208W
01:55: Berlin: -400W, Mumbai: -184W
02:00: Berlin: -400W, Mumbai: -158W
02:05: Berlin: -400W, Mumbai: -130W
02:10: Berlin: -400W, Mumbai: -100W
02:15: Berlin: -400W, Mumbai

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43200/43200 [00:00<00:00, 128050.80steps/s]
2025-07-06 15:24:40.145 | INFO     | mosaik.async_scenario:run:753 - Simulation finished successfully.


07:15: Berlin: 469W, Mumbai: 705W
07:20: Berlin: 457W, Mumbai: 693W
07:25: Berlin: 466W, Mumbai: 691W
07:30: Berlin: 490W, Mumbai: 699W
07:35: Berlin: 522W, Mumbai: 711W
07:40: Berlin: 556W, Mumbai: 723W
07:45: Berlin: 592W, Mumbai: 736W
07:50: Berlin: 642W, Mumbai: 758W
07:55: Berlin: 704W, Mumbai: 794W
08:00: Berlin: 775W, Mumbai: 842W
08:05: Berlin: 840W, Mumbai: 871W
08:10: Berlin: 885W, Mumbai: 852W
08:15: Berlin: 915W, Mumbai: 788W
08:20: Berlin: 941W, Mumbai: 731W
08:25: Berlin: 964W, Mumbai: 724W
08:30: Berlin: 985W, Mumbai: 766W
08:35: Berlin: 1006W, Mumbai: 821W
08:40: Berlin: 1028W, Mumbai: 851W
08:45: Berlin: 1049W, Mumbai: 853W
08:50: Berlin: 1069W, Mumbai: 844W
08:55: Berlin: 1086W, Mumbai: 831W
09:00: Berlin: 1101W, Mumbai: 816W
09:05: Berlin: 1117W, Mumbai: 781W
09:10: Berlin: 1132W, Mumbai: 718W
09:15: Berlin: 1148W, Mumbai: 628W
09:20: Berlin: 1163W, Mumbai: 542W
09:25: Berlin: 1176W, Mumbai: 488W
09:30: Berlin: 1188W, Mumbai: 465W
09:35: Berlin: 1200W, Mumbai: 456W
0

The controller monitored both datacenters and reported when their power consumption differed significantly.

## Summary

This tutorial showed how to create controllers that coordinate multiple microgrids:

1. **Multi-Microgrid Access**: Controllers receive states from all managed microgrids
2. **Simple Coordination**: Even basic monitoring can provide valuable insights
3. **Easy Setup**: Use `environment.add_controller(controller, [microgrid1, microgrid2])`

Controllers enable sophisticated coordination strategies for distributed systems, smart cities, and microgrid clusters.